In [78]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.0'
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [79]:
# Import packages
from pyspark.sql import SparkSession
import time



In [80]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [81]:
# Create a Spark session
spark = SparkSession.builder.appName("S3ReadExample").getOrCreate()

# Add the S3 file to SparkContext
spark.sparkContext.addFile(url)

# Read the CSV file from SparkFiles
home_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)

# Show the DataFrame
home_df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [82]:
# 2. Create a temporary view of the DataFrame.

home_df.createOrReplaceTempView("home_sales_view")

In [83]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

result_df_1 = spark.sql("""
    SELECT
        YEAR(date) AS Year,
        ROUND(AVG(Price), 2) AS AveragePrice

    FROM
        home_sales_view
    WHERE
        bedrooms = 4
    GROUP BY
        Year
    ORDER BY
        Year
""")

result_df_1.show()


+----+------------+
|Year|AveragePrice|
+----+------------+
|2019|    300263.7|
|2020|   298353.78|
|2021|   301819.44|
|2022|   296363.88|
+----+------------+



In [86]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
result_df_2 = spark.sql("""
    SELECT
        YEAR(date) AS Year,
        ROUND(AVG(Price), 2) AS AveragePrice
    FROM
        home_sales_view
    WHERE
        bedrooms = 3
        AND
        bathrooms = 3

    GROUP BY
        Year
    ORDER BY
        Year
""")

result_df_2.show()


+----+------------+
|Year|AveragePrice|
+----+------------+
|2019|   287287.82|
|2020|   294204.16|
|2021|   294211.46|
|2022|   292725.69|
+----+------------+



In [87]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
result_df_3 = spark.sql("""
    SELECT
        YEAR(date) AS Year,
        ROUND(AVG(Price), 2) AS AveragePrice
    FROM
        home_sales_view
    WHERE
        bedrooms = 3
        AND
        bathrooms = 3
        AND
        floors = 2
        AND
        sqft_living >= 2000

    GROUP BY
        Year
    ORDER BY
        Year
""")

result_df_3.show()


+----+------------+
|Year|AveragePrice|
+----+------------+
|2019|   289859.14|
|2020|   292289.09|
|2021|   296330.96|
|2022|   290242.99|
+----+------------+



In [88]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

result_df_4 = spark.sql("""
    SELECT
        ROUND(AVG(view), 2) AS Avg_View_Rating
    FROM
        home_sales_view
    WHERE
        Price >= 350000
""")

result_df_4.show()

end_time = time.time()
runtime = end_time - start_time

print("--- %s seconds ---" % (time.time() - start_time))

+---------------+
|Avg_View_Rating|
+---------------+
|          32.26|
+---------------+

--- 0.38787341117858887 seconds ---


In [70]:
# 7. Cache the the temporary table home_sales.
spark.sql("CACHE TABLE home_sales_view")

DataFrame[]

In [71]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales_view')

True

In [89]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

result_df_5 = spark.sql("""
    SELECT
        ROUND(AVG(view), 2) AS Avg_View_Rating
    FROM
        home_sales_view
    WHERE
        Price >= 350000
""")

result_df_5.show()

end_time = time.time()
runtime = end_time - start_time


print("--- %s seconds ---" % (time.time() - start_time))


+---------------+
|Avg_View_Rating|
+---------------+
|          32.26|
+---------------+

--- 0.14560246467590332 seconds ---


In [90]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data


home_df.write.partitionBy("date_built").mode("overwrite").parquet("home_partitioned")

In [91]:
# 11. Read the parquet formatted data.

# Read the partitioned Parquet data
partitioned_df = spark.read.parquet("home_partitioned")

# Perform a query to count records in each partition
p_result_df = partitioned_df.groupBy("date_built").count()

# Show the result DataFrame
p_result_df.show()

+----------+-----+
|date_built|count|
+----------+-----+
|      2015| 4248|
|      2014| 4163|
|      2012| 4181|
|      2017| 4237|
|      2013| 4085|
|      2016| 4128|
|      2010| 4097|
|      2011| 4148|
+----------+-----+



In [106]:
# 12. Create a temporary table for the parquet data.

# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

# Create a temporary view for the Parquet data
partitioned_df.createOrReplaceTempView("home_partitioned_view")

# Perform a SQL query on the temporary table

start_time = time.time()

result_df_6 = spark.sql("""
    SELECT
        ROUND(AVG(view), 2) AS Avg_View_Rating
    FROM
        home_partitioned_view
    WHERE
        Price >= 350000
""")

result_df_6.show()

end_time = time.time()
runtime = end_time - start_time

print("--- %s seconds ---" % (time.time() - start_time))

+---------------+
|Avg_View_Rating|
+---------------+
|          32.26|
+---------------+

--- 0.3580515384674072 seconds ---


In [107]:
# 14. Uncache the home_sales temporary table.

# Create a Spark session
spark = SparkSession.builder.appName("UncacheTableExample").getOrCreate()

# Uncache the "home_sales_view" temporary table
spark.catalog.uncacheTable("home_sales_view")

In [108]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached('home_sales_view')

False

In [109]:
#stop spark session
spark.stop

<bound method SparkSession.stop of <pyspark.sql.session.SparkSession object at 0x7dda337d24a0>>